# Go through the each cell.. used Chatgpt to understood each cell..

# Mount Drive

In [14]:
# @title
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#                                     1. Imports and Utilities

In [15]:
# @title
!pip install kornia torchmetrics scikit-image tqdm
!pip install kornia torchmetrics
!pip install torchmetrics
!pip install scikit-image torchvision
!pip install scikit-image
!pip install lpips
!pip install torchviz graphviz
!pip install kornia

In [16]:
# @title
# =========================================================
# Imports for Deep Learning and Computer Vision
# =========================================================
import os
import csv
import json
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms
from torchvision.utils import save_image
import torchvision.models as models
from torchvision.models import vgg16 # Specifically imported for potential feature extraction

import kornia
from kornia.color import rgb_to_hsv

# Metrics
from torchmetrics.image import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
from skimage.metrics import structural_similarity as ssim_sk, peak_signal_noise_ratio as psnr_sk

from tqdm import tqdm # For progress bars

# 2. Dataset: Paired Distorted / Clean Images

In [17]:
# @title
# =========================================================
# 2. Dataset: ImagePairDataset (Fixed Joint Transform Seed)
# =========================================================
class ImagePairDataset(Dataset):
    """
    Paired dataset for distorted -> clean image reconstruction.
    Returns: (dist_tensor, clean_tensor, path_str)
    """
    # noise image subtitle:
    # Dataset / Noise Folder / open a image / click three dot
    #  #  #  #
    def __init__(self, distorted_dir, clean_dir, transform=None,
                 dist_suffix='_noise4_multiply_strong', clean_suffix='_original'): # e.g _gaussian
        self.distorted_dir = distorted_dir
        self.clean_dir = clean_dir
        self.transform = transform
        self.dist_suffix = dist_suffix
        self.clean_suffix = clean_suffix

        distorted_filenames = sorted([
            f for f in os.listdir(distorted_dir)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ])

        print(f"📁 Found {len(distorted_filenames)} distorted images in {distorted_dir}")
        self.data_pairs = []
        for dist_file in distorted_filenames:
            clean_file = dist_file.replace(dist_suffix, clean_suffix)
            dist_path = os.path.join(distorted_dir, dist_file)
            clean_path = os.path.join(clean_dir, clean_file)
            if os.path.exists(clean_path):
                self.data_pairs.append((dist_path, clean_path))
            else:
                print(f"   ⚠️ Clean not found for {dist_file}")

        if not self.data_pairs:
            raise FileNotFoundError("No valid image pairs found!")

        print(f"✅ Dataset ready with {len(self.data_pairs)} image pairs")

    def __len__(self):
        return len(self.data_pairs)

    def __getitem__(self, idx):
        dist_path, clean_path = self.data_pairs[idx]
        dist_img = Image.open(dist_path).convert("RGB")
        clean_img = Image.open(clean_path).convert("RGB")

        # Apply same random seed for paired augmentations to keep alignment
        seed = np.random.randint(2147483647)
        torch.manual_seed(seed)
        if self.transform:
            dist_tensor = self.transform(dist_img)
        else:
            dist_tensor = transforms.ToTensor()(dist_img)

        torch.manual_seed(seed)
        if self.transform:
            clean_tensor = self.transform(clean_img)
        else:
            clean_tensor = transforms.ToTensor()(clean_img)

        return dist_tensor, clean_tensor, dist_path

# 3. Model: Attention Residual U-Net (ARU-Net)

In [18]:
# @title
# =========================================================
# 3. Model: Attention Residual U-Net (ARU-Net)
# =========================================================
class ResidualBlock(nn.Module): # thing about it to renamed. we maybe changed it.
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

    def forward(self, x):
        res = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        return self.relu(out + res)


class AttentionBlock(nn.Module): # we can change it to make unique.
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Sequential(nn.Conv2d(F_g, F_int, kernel_size=1), nn.BatchNorm2d(F_int))
        self.W_x = nn.Sequential(nn.Conv2d(F_l, F_int, kernel_size=1), nn.BatchNorm2d(F_int))
        self.psi = nn.Sequential(nn.Conv2d(F_int, 1, kernel_size=1), nn.BatchNorm2d(1), nn.Sigmoid())
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        psi = self.relu(self.W_g(g) + self.W_x(x))
        psi = self.psi(psi)
        return x * psi


class AttentionResidualUNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = nn.Sequential(nn.Conv2d(3, 64, 3, 1, 1), nn.ReLU(True),
                                  nn.Conv2d(64, 64, 3, 1, 1), nn.ReLU(True),
                                  ResidualBlock(64))
        self.pool1 = nn.MaxPool2d(2)

        self.enc2 = nn.Sequential(nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(True),
                                  nn.Conv2d(128, 128, 3, 1, 1), nn.ReLU(True),
                                  ResidualBlock(128))
        self.pool2 = nn.MaxPool2d(2)

        self.enc3 = nn.Sequential(nn.Conv2d(128, 256, 3, 1, 1), nn.ReLU(True),
                                  nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(True),
                                  ResidualBlock(256))
        self.pool3 = nn.MaxPool2d(2)

        self.enc4 = nn.Sequential(nn.Conv2d(256, 512, 3, 1, 1), nn.ReLU(True),
                                  nn.Conv2d(512, 512, 3, 1, 1), nn.ReLU(True),
                                  ResidualBlock(512))
        self.pool4 = nn.MaxPool2d(2)

        # Center
        self.center = nn.Sequential(nn.Conv2d(512, 1024, 3, 1, 1), nn.ReLU(True), ResidualBlock(1024))

        # Attention gates
        self.att4 = AttentionBlock(512, 512, 256)
        self.att3 = AttentionBlock(256, 256, 128)
        self.att2 = AttentionBlock(128, 128, 64)
        self.att1 = AttentionBlock(64, 64, 32)

        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = nn.Sequential(nn.Conv2d(1024, 512, 3, 1, 1), nn.ReLU(True), ResidualBlock(512))

        self.up3 = nn.ConvTranspose2d(512, 256, 2, 2)
        self.dec3 = nn.Sequential(nn.Conv2d(512, 256, 3, 1, 1), nn.ReLU(True), ResidualBlock(256))

        self.up2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = nn.Sequential(nn.Conv2d(256, 128, 3, 1, 1), nn.ReLU(True), ResidualBlock(128))

        self.up1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = nn.Sequential(nn.Conv2d(128, 64, 3, 1, 1), nn.ReLU(True), ResidualBlock(64))

        self.out = nn.Conv2d(64, 3, kernel_size=1)

    def forward(self, x):
        e1 = self.enc1(x); p1 = self.pool1(e1)
        e2 = self.enc2(p1); p2 = self.pool2(e2)
        e3 = self.enc3(p2); p3 = self.pool3(e3)
        e4 = self.enc4(p3); p4 = self.pool4(e4)
        c = self.center(p4)

        d4 = self.up4(c)
        e4_att = self.att4(d4, e4)
        d4 = self.dec4(torch.cat([d4, e4_att], dim=1))

        d3 = self.up3(d4)
        e3_att = self.att3(d3, e3)
        d3 = self.dec3(torch.cat([d3, e3_att], dim=1))

        d2 = self.up2(d3)
        e2_att = self.att2(d2, e2)
        d2 = self.dec2(torch.cat([d2, e2_att], dim=1))

        d1 = self.up1(d2)
        e1_att = self.att1(d1, e1)
        d1 = self.dec1(torch.cat([d1, e1_att], dim=1))

        return torch.sigmoid(self.out(d1))

# 4. Mask Utilities leaf mask from clean image

In [20]:
# @title
# =========================================================
# 4. Mask Utilities (Leaf Mask from Clean Image)
# =========================================================
def create_leaf_mask_from_rgb_tensor(img_rgb, low_h=0.18, high_h=0.45, min_sat=0.15, min_val=0.05):
    """
    Create a soft binary mask for leaf regions from an RGB tensor.
    - img_rgb: (B,3,H,W), values in [0,1]
    - Hues selected around green (default approx 0.18..0.45 in 0..1)
    Returns: mask (B,1,H,W) float in {0,1}
    """
    # convert to HSV using kornia (expects float tensor 0..1)
    hsv = rgb_to_hsv(img_rgb)  # returns (B,3,H,W): h,s,v in [0,1]
    h = hsv[:, 0:1, :, :]  # (B,1,H,W)
    s = hsv[:, 1:2, :, :]
    v = hsv[:, 2:3, :, :]

    # handle hue wrap-around (not needed for green but robust)
    # keep hue between low_h and high_h
    mask_h = (h >= low_h) & (h <= high_h)
    mask_sv = (s >= min_sat) & (v >= min_val)
    mask = (mask_h & mask_sv).float()

    # optional: morphological smoothing (Gaussian blur) to reduce noisy tiny bits
    # Use kornia median blur with small kernel
    try:
        mask = kornia.filters.median_blur(mask, (3, 3))
    except Exception:
        # fallback: no filtering if kornia version differs
        pass

    return mask  # (B,1,H,W)

# 5. Loss: Combined (Perceptual + SSIM + Edge + L1)

In [21]:
# @title
# =========================================================
# 5. Loss: Combined (Perceptual + L1 + Edge + Masked SSIM)
# =========================================================
class CombinedLoss(nn.Module):
    def __init__(self, device='cpu', use_vgg=True):
        super().__init__()
        self.device = device
        self.use_vgg = use_vgg
        if use_vgg:
            vgg = models.vgg16(pretrained=True).features[:16].eval().to(device)
            for p in vgg.parameters():
                p.requires_grad = False
            self.vgg = vgg
        self.ssim = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
        self.laplace = kornia.filters.Laplacian(3)

    def forward(self, output, target, mask=None):
        output = torch.clamp(output, 0.0, 1.0)
        target = torch.clamp(target, 0.0, 1.0)
        if mask is None:
            with torch.no_grad():
                mask = create_leaf_mask_from_rgb_tensor(target)  # (B,1,H,W)
        if mask.dim() == 3:
            mask = mask.unsqueeze(1)
        if mask.shape[1] == 1 and output.shape[1] == 3:
            mask_rgb = mask.repeat(1, 3, 1, 1)
        else:
            mask_rgb = mask
        content_loss = F.l1_loss(self.vgg(output), self.vgg(target)) if self.use_vgg else 0.0
        pixel_loss = F.l1_loss(output, target)
        edge_loss = F.l1_loss(self.laplace(output), self.laplace(target))
        output_masked = output * mask_rgb
        target_masked = target * mask_rgb
        if mask_rgb.sum() <= 1e-6:
            ssim_value = self.ssim(output, target).mean()
        else:
            ssim_value = self.ssim(output_masked, target_masked).mean()

        ssim_loss = 1.0 - ssim_value

        total = 0.4 * pixel_loss + 0.25 * content_loss + 0.2 * edge_loss + 0.15 * ssim_loss
        return total

# 6. Training Function

In [22]:
# @title
# =========================================================
# 6. Training Function (with epoch-wise CSV logging)
# =========================================================
def train(model, train_loader, val_loader, device, epochs=70, lr=1e-4, optimizer=None, scheduler=None, save_dir="./Output"):
    print("🚀 Starting Training (Masked SSIM)...")
    os.makedirs(save_dir, exist_ok=True)
    csv_path = os.path.join(save_dir, "training_validation_metrics.csv")

    criterion = CombinedLoss(device=device, use_vgg=True)
    if optimizer is None:
        optimizer = optim.Adam(model.parameters(), lr=lr)
    if scheduler is None:
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    scaler = torch.cuda.amp.GradScaler() if 'cuda' in device else None

    # Initialize CSV file
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "train_loss", "train_psnr", "train_ssim", "val_loss", "val_psnr", "val_ssim"])

    model.train()
    train_losses = []

    for epoch in range(epochs):
        print(f"\n📈 Epoch {epoch+1}/{epochs}")
        running_loss = 0.0
        skipped_batches = 0

        # ================= Training ==================
        for batch_idx, batch in enumerate(train_loader):
            if len(batch) == 3:
                dist_img, clean_img, _ = batch
            else:
                dist_img, clean_img = batch
            dist_img, clean_img = dist_img.to(device), clean_img.to(device)

            if dist_img.shape != clean_img.shape:
                skipped_batches += 1
                continue

            optimizer.zero_grad()
            if scaler:
                with torch.amp.autocast('cuda'):
                    output = model(dist_img)
                    if output.shape != clean_img.shape:
                        output = F.interpolate(output, size=clean_img.shape[2:], mode='bilinear', align_corners=False)
                    loss = criterion(output, clean_img, None)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                output = model(dist_img)
                if output.shape != clean_img.shape:
                    output = F.interpolate(output, size=clean_img.shape[2:], mode='bilinear', align_corners=False)
                loss = criterion(output, clean_img, None)
                loss.backward()
                optimizer.step()

            running_loss += loss.item()
            if batch_idx % 10 == 0:
                print(f"   Batch {batch_idx:4d}, Loss: {loss.item():.6f}")

        scheduler.step()
        avg_train_loss = running_loss / (len(train_loader) - skipped_batches)
        train_losses.append(avg_train_loss)

        # ================= Validation ==================
        model.eval()
        val_loss, val_psnr_list, val_ssim_list = 0.0, [], []

        with torch.no_grad():
            for val_batch in val_loader:
                if len(val_batch) == 3:
                    dist_img, clean_img, _ = val_batch
                else:
                    dist_img, clean_img = val_batch
                dist_img, clean_img = dist_img.to(device), clean_img.to(device)

                output = model(dist_img)
                if output.shape != clean_img.shape:
                    output = F.interpolate(output, size=clean_img.shape[2:], mode='bilinear', align_corners=False)
                loss = criterion(output, clean_img, None)
                val_loss += loss.item()

                output_np = output.detach().cpu().numpy().transpose(0, 2, 3, 1)
                target_np = clean_img.cpu().numpy().transpose(0, 2, 3, 1)

                for i in range(output_np.shape[0]):
                    pred = np.clip(output_np[i], 0, 1)
                    target = np.clip(target_np[i], 0, 1)
                    val_psnr_list.append(psnr_sk(target, pred, data_range=1.0))
                    val_ssim_list.append(ssim_sk(target, pred, channel_axis=-1, data_range=1.0))

        avg_val_loss = val_loss / len(val_loader)
        avg_val_psnr = np.mean(val_psnr_list)
        avg_val_ssim = np.mean(val_ssim_list)

        # Compute PSNR/SSIM on train (small sample)
        train_psnr_list, train_ssim_list = [], []
        with torch.no_grad():
            for dist_img, clean_img, _ in train_loader:
                dist_img, clean_img = dist_img.to(device), clean_img.to(device)
                output = model(dist_img)
                output_np = output.detach().cpu().numpy().transpose(0, 2, 3, 1)
                target_np = clean_img.cpu().numpy().transpose(0, 2, 3, 1)
                for i in range(output_np.shape[0]):
                    train_psnr_list.append(psnr_sk(target_np[i], output_np[i], data_range=1.0))
                    train_ssim_list.append(ssim_sk(target_np[i], output_np[i], channel_axis=-1, data_range=1.0))
                break  # only one batch for speed
        avg_train_psnr = np.mean(train_psnr_list)
        avg_train_ssim = np.mean(train_ssim_list)

        # Save metrics to CSV
        with open(csv_path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                epoch + 1,
                avg_train_loss,
                avg_train_psnr,
                avg_train_ssim,
                avg_val_loss,
                avg_val_psnr,
                avg_val_ssim
            ])

        print(f"✅ Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | "
              f"Train PSNR: {avg_train_psnr:.2f} | Val PSNR: {avg_val_psnr:.2f}")

        model.train()  # set back to train mode

    print(f"\n✅ Training Complete! Metrics saved to {csv_path}")
    return train_losses


# 7. Evaluation, Metrics Saving & Visualization Utilities


In [23]:
# @title
# =========================================================
# 7. Evaluation, Metrics Saving & Visualization - FIXED (win_size + scoping issues)
# =========================================================
def evaluate(model, dataloader, device, save_dir="./Output"):
    print("\n🧪 Starting Evaluation (masked SSIM for each image)...")
    os.makedirs(save_dir, exist_ok=True)
    csv_path = os.path.join(save_dir, "eval_metricsmetrics.csv")

    model.eval()
    psnr_scores, ssim_scores = [], []
    csv_rows = []

    with torch.no_grad():
        idx_counter = 0
        for batch_idx, batch in enumerate(dataloader):
            if len(batch) == 3:
                dist_img, clean_img, paths = batch
            else:
                dist_img, clean_img = batch
                paths = [f"sample_{i}" for i in range(dist_img.shape[0])]

            dist_img, clean_img = dist_img.to(device), clean_img.to(device)

            # Forward pass (with AMP if CUDA)
            if 'cuda' in device:
                with torch.amp.autocast('cuda'):
                    output = model(dist_img)
            else:
                output = model(dist_img)

            # Resize and clamp
            if output.shape != clean_img.shape:
                output = torch.nn.functional.interpolate(output, size=clean_img.shape[2:], mode='bilinear', align_corners=False)
            output = torch.clamp(output, 0, 1)

            # Convert to numpy for metrics
            output_np = output.cpu().numpy().transpose(0, 2, 3, 1)
            clean_np = clean_img.cpu().numpy().transpose(0, 2, 3, 1)
            mask_tensor = create_leaf_mask_from_rgb_tensor(clean_img)
            mask_np = mask_tensor.cpu().numpy().transpose(0, 2, 3, 1)

            for i in range(output_np.shape[0]):
                pred = np.clip(output_np[i], 0, 1)
                target = np.clip(clean_np[i], 0, 1)
                mask_i = (mask_np[i, :, :, 0] > 0.5).astype(np.float32)

                # Compute PSNR + SSIM (masked or global)
                if mask_i.sum() < 49:
                    psnr_val = psnr_sk(target, pred, data_range=1.0)
                    ssim_val = ssim_sk(target, pred, channel_axis=-1, data_range=1.0)
                else:
                    ys, xs = np.where(mask_i)
                    y0, y1 = ys.min(), ys.max()
                    x0, x1 = xs.min(), xs.max()
                    crop_h, crop_w = y1 - y0 + 1, x1 - x0 + 1

                    if crop_h < 7 or crop_w < 7:
                        psnr_val = psnr_sk(target, pred, data_range=1.0)
                        ssim_val = ssim_sk(target, pred, channel_axis=-1, data_range=1.0)
                    else:
                        pred_crop = pred[y0:y1+1, x0:x1+1, :]
                        target_crop = target[y0:y1+1, x0:x1+1, :]
                        win_size = min(7, min(crop_h, crop_w))
                        if win_size % 2 == 0: win_size -= 1
                        if win_size < 3: win_size = 3
                        psnr_val = psnr_sk(target_crop, pred_crop, data_range=1.0)
                        try:
                            ssim_val = ssim_sk(target_crop, pred_crop, channel_axis=-1, data_range=1.0, win_size=win_size)
                        except ValueError:
                            ssim_val = ssim_sk(target, pred, channel_axis=-1, data_range=1.0)

                psnr_scores.append(psnr_val)
                ssim_scores.append(ssim_val)
                csv_rows.append([idx_counter, psnr_val, ssim_val])

                # ✅ Save reconstructed image only
                recon_name = f"Image_{idx_counter}_Reconstructed.jpg"
                save_path = os.path.join(save_dir, recon_name)
                save_image(torch.from_numpy(pred.transpose(2, 0, 1)), save_path)

                idx_counter += 1

            if batch_idx % 5 == 0:
                print(f"   Processed batch {batch_idx}")

    # Write CSV
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Index", "PSNR", "SSIM"])
        writer.writerows(csv_rows)

    if len(psnr_scores) == 0:
        print("⚠️ No valid samples evaluated!")
        return 0.0, 0.0, [], []

    print(f"\n📊 Evaluation Results:")
    print(f"   PSNR (mean ± std):  {np.mean(psnr_scores):.4f} ± {np.std(psnr_scores):.4f}")
    print(f"   SSIM (mean ± std):  {np.mean(ssim_scores):.4f} ± {np.std(ssim_scores):.4f}")
    print(f"   Saved {len(psnr_scores)} reconstructed images and metrics to: {csv_path}")

    return np.mean(psnr_scores), np.mean(ssim_scores), psnr_scores, ssim_scores


def visualize_training_progress(train_losses):
    if not train_losses:
        print("⚠️ No training losses to visualize!")
        return
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, linewidth=2)
    plt.title('Training Loss Progress', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)
    plt.show()
    print(f"📈 Initial Loss: {train_losses[0]:.6f}, Final Loss: {train_losses[-1]:.6f}")


def visualize_sample_results(model, dataloader, device, num_samples=3, save_dir=None):
    model.eval()
    samples_collected = 0
    with torch.no_grad():
        for batch in dataloader:  # No batch_idx here; use samples_collected instead
            if len(batch) == 3:
                dist_img, clean_img, paths = batch
            else:
                dist_img, clean_img = batch
                paths = [f"sample_{i}" for i in range(dist_img.shape[0])]

            dist_img, clean_img = dist_img.to(device), clean_img.to(device)
            output = model(dist_img)
            if output.shape != clean_img.shape:
                output = torch.nn.functional.interpolate(output, size=clean_img.shape[2:], mode='bilinear', align_corners=False)

            # Compute masks for visualization (inside this function)
            mask_tensor = create_leaf_mask_from_rgb_tensor(clean_img)  # (B,1,H,W)
            mask_np = mask_tensor.cpu().numpy().transpose(0, 2, 3, 1)  # (B,H,W,1)

            for i in range(dist_img.shape[0]):
                if samples_collected >= num_samples:
                    return

                dist_np = dist_img[i].cpu().numpy().transpose(1, 2, 0)
                out_np = output[i].cpu().numpy().transpose(1, 2, 0)
                clean_np = clean_img[i].cpu().numpy().transpose(1, 2, 0)

                pred = np.clip(out_np, 0, 1)
                target = np.clip(clean_np, 0, 1)

                # Compute mask_i for this sample (for debug print/viz)
                mask_i = (mask_np[i, :, :, 0] > 0.5).astype(np.float32)

                plt.figure(figsize=(12, 4))
                plt.subplot(1, 3, 1); plt.imshow(np.clip(dist_np, 0, 1)); plt.title('Distorted'); plt.axis('off')
                plt.subplot(1, 3, 2); plt.imshow(pred); plt.title('Reconstructed'); plt.axis('off')
                plt.subplot(1, 3, 3); plt.imshow(target); plt.title('Original'); plt.axis('off')
                plt.tight_layout(); plt.show()

                if save_dir:
                    os.makedirs(save_dir, exist_ok=True)
                    filename = os.path.basename(paths[i]) if isinstance(paths[i], str) else f"sample_{samples_collected}.png"
                    recon_path = os.path.join(save_dir, f"recon_{filename}")
                    plt.imsave(recon_path, pred)
                    print(f"   Saved: {recon_path}")

                # DEBUG: Visualize mask and print area for first sample only
                if samples_collected == 0:
                    plt.figure(figsize=(6, 6))
                    plt.imshow(mask_np[i, :, :, 0], cmap='gray')
                    plt.title('Leaf Mask (White = Leaf Region)')
                    plt.axis('off')
                    plt.show()
                    print(f"Mask area (pixels): {mask_i.sum()}")  # Print size

                samples_collected += 1


def save_training_results(train_losses, psnr_score, ssim_score, filename="training_results.json"):
    if not train_losses:
        print("⚠️ No results to save!")
        return
    results = {
        'train_losses': [float(loss) for loss in train_losses],
        'final_psnr': float(psnr_score),
        'final_ssim': float(ssim_score),
        'epochs': len(train_losses)
    }
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"✅ Training results saved to {filename}")


def save_metrics_csv(model, dataloader, device, save_path):
    model.eval()
    records = []
    with torch.no_grad():
        for batch in dataloader:
            if len(batch) == 3:
                dist_img, clean_img, paths = batch
            else:
                dist_img, clean_img = batch
                paths = [f"sample_{i}" for i in range(dist_img.shape[0])]

            dist_img, clean_img = dist_img.to(device), clean_img.to(device)
            output = model(dist_img)
            if output.shape != clean_img.shape:
                output = torch.nn.functional.interpolate(output, size=clean_img.shape[2:], mode='bilinear', align_corners=False)

            # Convert to numpy once per batch
            output_np = output.cpu().numpy().transpose(0, 2, 3, 1)
            clean_np = clean_img.cpu().numpy().transpose(0, 2, 3, 1)

            # Create masks (add this for masked metrics)
            mask_tensor = create_leaf_mask_from_rgb_tensor(clean_img)  # (B,1,H,W)
            mask_np = mask_tensor.cpu().numpy().transpose(0, 2, 3, 1)  # (B,H,W,1)

            for i in range(dist_img.shape[0]):
                pred = np.clip(output_np[i], 0, 1)
                target = np.clip(clean_np[i], 0, 1)
                mask_i = (mask_np[i, :, :, 0] > 0.5).astype(np.float32)  # binary HxW

                # If mask too small, fallback to global
                if mask_i.sum() < 49:  # 7x7 min
                    psnr_val = psnr_sk(target, pred, data_range=1.0)
                    ssim_val = ssim_sk(target, pred, channel_axis=-1, data_range=1.0)  # Global, no issue
                else:
                    ys, xs = np.where(mask_i)
                    if ys.size == 0:
                        psnr_val = psnr_sk(target, pred, data_range=1.0)
                        ssim_val = ssim_sk(target, pred, channel_axis=-1, data_range=1.0)
                    else:
                        y0, y1 = ys.min(), ys.max()
                        x0, x1 = xs.min(), xs.max()
                        crop_h, crop_w = y1 - y0 + 1, x1 - x0 + 1
                        if crop_h < 7 or crop_w < 7:
                            psnr_val = psnr_sk(target, pred, data_range=1.0)
                            ssim_val = ssim_sk(target, pred, channel_axis=-1, data_range=1.0)
                        else:
                            pred_crop = pred[y0:y1+1, x0:x1+1, :]
                            target_crop = target[y0:y1+1, x0:x1+1, :]

                            # Dynamic win_size
                            win_size = min(7, min(crop_h, crop_w))
                            if win_size % 2 == 0: win_size -= 1
                            if win_size < 3: win_size = 3

                            psnr_val = psnr_sk(target_crop, pred_crop, data_range=1.0)
                            try:
                                ssim_val = ssim_sk(target_crop, pred_crop, channel_axis=-1, data_range=1.0, win_size=win_size)
                            except ValueError:
                                ssim_val = ssim_sk(target, pred, channel_axis=-1, data_range=1.0)

                records.append({
                    "filename": os.path.basename(paths[i]) if isinstance(paths[i], str) else f"sample_{i}",
                    "PSNR": float(psnr_val),
                    "SSIM": float(ssim_val)
                })
    df = pd.DataFrame(records)
    df.to_csv(save_path, index=False)
    print(f"Metrics saved to {save_path}")
    return df

# 8. Main Execution Pipeline (Example)

In [25]:
# @title
# =========================================================
# 8. Main Execution – 80/10/10 + FULL-RES RECONSTRUCTIONS (ALL SPLITS)
# =========================================================
if __name__ == "__main__":
    # --------------------------------------------------------------
    # 0. Device
    # --------------------------------------------------------------
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # --------------------------------------------------------------
    # 1. Paths & folders
    # --------------------------------------------------------------
    dataset_name      = "Noise_Multiply_Strong" #

    save_model_folder = "/content/drive/MyDrive/FrameReconstruction/Save_Model" # .pth model eta save korar jann

    distorted_dir     = "/content/drive/MyDrive/FrameReconstruction/dataset/Noise_Multiply_Strong" # noise folder

    clean_dir         = "/content/drive/MyDrive/FrameReconstruction/dataset/Orginal" # fixed eta change hobe na.

    custom_save_dir   = "/content/drive/MyDrive/FrameReconstruction/Output/Noise_Multiply_Strong" # Reconstructed all image will saved here

    os.makedirs(save_model_folder, exist_ok=True)
    os.makedirs(custom_save_dir,   exist_ok=True)

    recon_all_dir = os.path.join(custom_save_dir, "reconstructed_all")
    samples_dir   = os.path.join(custom_save_dir, "samples")
    os.makedirs(recon_all_dir, exist_ok=True)
    os.makedirs(samples_dir,   exist_ok=True)

    # --------------------------------------------------------------
    # 2. Transforms (train: 224×224 patch | eval: full size)
    # --------------------------------------------------------------
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(5),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
    ])
    eval_transform = transforms.Compose([transforms.ToTensor()])

    # --------------------------------------------------------------
    # 3. Full dataset (no transform) + split
    # --------------------------------------------------------------
    full_dataset = ImagePairDataset(distorted_dir, clean_dir, transform=None)
    N = len(full_dataset)
    print(f"Total paired images: {N}")

    train_N = int(0.8 * N)
    val_N   = int(0.1 * N)
    test_N  = N - train_N - val_N

    torch.manual_seed(42)
    train_idx, val_idx, test_idx = torch.utils.data.random_split(
        range(N), [train_N, val_N, test_N]
    )

    # --------------------------------------------------------------
    # 4. Wrapper to apply correct transform per split
    # --------------------------------------------------------------
    class TransformDataset(torch.utils.data.Dataset):
        def __init__(self, base_dataset, indices, transform):
            self.base = base_dataset
            self.idx  = indices
            self.transform = transform

        def __len__(self): return len(self.idx)

        def __getitem__(self, i):
            idx = self.idx[i]
            dist_path, clean_path = self.base.data_pairs[idx]
            dist_img = Image.open(dist_path).convert("RGB")
            clean_img = Image.open(clean_path).convert("RGB")

            seed = np.random.randint(2147483647)
            torch.manual_seed(seed)
            dist_tensor = self.transform(dist_img) if self.transform else transforms.ToTensor()(dist_img)
            torch.manual_seed(seed)
            clean_tensor = self.transform(clean_img) if self.transform else transforms.ToTensor()(clean_img)

            return dist_tensor, clean_tensor, dist_path

    train_dataset = TransformDataset(full_dataset, train_idx.indices, train_transform)
    val_dataset   = TransformDataset(full_dataset, val_idx.indices,   eval_transform)
    test_dataset  = TransformDataset(full_dataset, test_idx.indices,  eval_transform)

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True,  num_workers=2)
    val_loader   = DataLoader(val_dataset,   batch_size=1, shuffle=False, num_workers=2)
    test_loader  = DataLoader(test_dataset,  batch_size=1, shuffle=False, num_workers=2)

    print(f"Train / Val / Test → {len(train_dataset)} / {len(val_dataset)} / {len(test_dataset)}")

    # --------------------------------------------------------------
    # 5. Model, optimizer, scheduler
    # --------------------------------------------------------------
    model = AttentionResidualUNet().to(device)
    print(f"Model params: {sum(p.numel() for p in model.parameters()):,}")

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)

    # --------------------------------------------------------------
    # 6. TRAIN (with early stopping)
    # --------------------------------------------------------------
    train_losses = train(
        model, train_loader, val_loader, device,
        epochs=150, optimizer=optimizer, scheduler=scheduler,
        save_dir=custom_save_dir, patience=20
    )

    # --------------------------------------------------------------
    # 7. SAVE ALL RECONSTRUCTIONS + PER-IMAGE METRICS (TRAIN + VAL + TEST)
    # --------------------------------------------------------------
    print("\n=== SAVING FULL-RES RECONSTRUCTIONS + METRICS FOR ALL SPLITS ===")

    def save_split_reconstructions(loader, split_name, recon_dir):
        csv_path = os.path.join(custom_save_dir, f"metrics_{split_name}.csv")
        rows = []
        model.eval()
        with torch.no_grad():
            for dist_img, clean_img, path in tqdm(loader, desc=f"[{split_name.upper()}]"):
                dist_img  = dist_img.to(device)
                clean_img = clean_img.to(device)

                out = model(dist_img)
                if out.shape[2:] != clean_img.shape[2:]:
                    out = F.interpolate(out, size=clean_img.shape[2:], mode='bilinear', align_corners=False)
                out = torch.clamp(out, 0, 1)

                for i in range(out.shape[0]):
                    pred_np  = out[i].cpu().numpy().transpose(1, 2, 0)
                    clean_np = clean_img[i].cpu().numpy().transpose(1, 2, 0)

                    psnr_val = psnr_sk(clean_np, pred_np, data_range=1.0)
                    ssim_val = ssim_sk(clean_np, pred_np, channel_axis=-1, data_range=1.0)

                    base = os.path.splitext(os.path.basename(path[i]))[0]
                    img_path = os.path.join(recon_dir, f"{base}_{split_name}_recon.png")
                    Image.fromarray((pred_np * 255).astype(np.uint8)).save(img_path)

                    rows.append([base, psnr_val, ssim_val, img_path])

        pd.DataFrame(rows, columns=["filename", "PSNR", "SSIM", "recon_path"]).to_csv(csv_path, index=False)
        print(f"{split_name.upper()} → {csv_path}")
        print(f"   Avg PSNR: {np.mean([r[1] for r in rows]):.3f} | Avg SSIM: {np.mean([r[2] for r in rows]):.3f}")

    # Run on all splits
    save_split_reconstructions(train_loader, "train", recon_all_dir)
    save_split_reconstructions(val_loader,   "val",   recon_all_dir)
    save_split_reconstructions(test_loader,  "test",  recon_all_dir)

    # --------------------------------------------------------------
    # 8. Optional: visualize a few samples
    # --------------------------------------------------------------
    visualize_sample_results(model, test_loader, device, num_samples=5, save_dir=samples_dir)

    # --------------------------------------------------------------
    # 9. Save final model
    # --------------------------------------------------------------
    final_path = os.path.join(save_model_folder, f"{dataset_name}_AttentionResidualUNet_final.pth")
    torch.save(model.state_dict(), final_path)
    print(f"\nFinal model saved → {final_path}")

    # --------------------------------------------------------------
    # DONE
    # --------------------------------------------------------------
    print("\nAll done! Check:")
    print(f"   • Reconstructed images → {recon_all_dir}")
    print(f"   • Metrics CSVs → {custom_save_dir}/metrics_*.csv")

Using device: cuda


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/FrameReconstruction/dataset/Noise_Multiply_Strong'

# Plot Random Sample with Metrices Overlay

In [ ]:
# @title
distorted_dir = "Noisy | Blur image Folder link"
clean_dir = "your Original image folder link"
custom_save_dir = "Your output folder link"

# Plot SSIM & PSNR

In [ ]:
# @title
# =========================================================
# Bonus: Visualize SSIM and PSNR Metrics from Saved CSVs
# =========================================================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

eval_csv_path = os.path.join(custom_save_dir, "eval_metrics.csv")  # From evaluate
per_image_csv_path = os.path.join(custom_save_dir, "metrics_per_image.csv")  # From save_metrics_csv
fig_save_path = os.path.join(custom_save_dir, "metrics_visualization.png")

# Set style for nicer plots
sns.set(style="whitegrid")
# ---------------------------

# Load data (use eval_metrics if available, fallback to per_image)
if os.path.exists(eval_csv_path):
    df = pd.read_csv(eval_csv_path)
    print(f"Loaded {len(df)} samples from {eval_csv_path}")
elif os.path.exists(per_image_csv_path):
    df = pd.read_csv(per_image_csv_path)
    print(f"Loaded {len(df)} samples from {per_image_csv_path} (fallback)")
else:
    raise FileNotFoundError(f"No metrics CSV found in {custom_save_dir}! Run evaluate/save_metrics_csv first.")

# Assume columns: 'Index', 'PSNR', 'SSIM' (adjust if needed, e.g., df['psnr'] = df['PSNR'])
if 'PSNR' not in df.columns or 'SSIM' not in df.columns:
    # Flexible column names
    psnr_col = next((col for col in df.columns if 'PSNR' in col.upper()), None)
    ssim_col = next((col for col in df.columns if 'SSIM' in col.upper()), None)
    if not psnr_col or not ssim_col:
        raise ValueError("CSV must have PSNR and SSIM columns!")
    df.rename(columns={psnr_col: 'PSNR', ssim_col: 'SSIM'}, inplace=True)

psnr_values = df['PSNR'].values
ssim_values = df['SSIM'].values

# Compute stats
print("\n📊 Metrics Summary:")
print(f"PSNR: mean={np.mean(psnr_values):.4f}, std={np.std(psnr_values):.4f}, min={np.min(psnr_values):.4f}, max={np.max(psnr_values):.4f}")
print(f"SSIM: mean={np.mean(ssim_values):.4f}, std={np.std(ssim_values):.4f}, min={np.min(ssim_values):.4f}, max={np.max(ssim_values):.4f}")

# Create figure with subplots
fig = plt.figure(figsize=(16, 10))

# 1. Histogram PSNR
ax1 = fig.add_subplot(2, 2, 1)
sns.histplot(psnr_values, bins=20, kde=True, color='blue', ax=ax1)
ax1.set_title('PSNR Distribution (Histogram + KDE)')
ax1.set_xlabel('PSNR (dB)')
ax1.set_ylabel('Frequency')

# 2. Histogram SSIM
ax2 = fig.add_subplot(2, 2, 2)
sns.histplot(ssim_values, bins=20, kde=True, color='green', ax=ax2)
ax2.set_title('SSIM Distribution (Histogram + KDE)')
ax2.set_xlabel('SSIM')
ax2.set_ylabel('Frequency')

# 3. Scatter PSNR vs SSIM
ax3 = fig.add_subplot(2, 2, 3)
sns.scatterplot(x=ssim_values, y=psnr_values, alpha=0.6, color='purple', ax=ax3)
ax3.set_title('PSNR vs SSIM (Correlation)')
ax3.set_xlabel('SSIM')
ax3.set_ylabel('PSNR (dB)')
# Add correlation coefficient
corr = np.corrcoef(ssim_values, psnr_values)[0, 1]
ax3.text(0.05, 0.95, f'Corr: {corr:.3f}', transform=ax3.transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))

# 4. Box plots
ax4 = fig.add_subplot(2, 2, 4)
metrics_melted = pd.melt(df[['PSNR', 'SSIM']], var_name='Metric', value_name='Value')
sns.boxplot(x='Metric', y='Value', data=metrics_melted, palette=['blue', 'green'], ax=ax4)
ax4.set_title('PSNR and SSIM Box Plots (Summary)')

plt.tight_layout()
plt.savefig(fig_save_path, dpi=150, bbox_inches='tight')
print(f"Visualization saved to {fig_save_path}")
plt.show()

# 3. Training Loss + LR Curve (Over Epochs)

In [ ]:
# @title
# =========================================================
# Viz 3: Training Curves (Loss + Learning Rate)
# =========================================================
import json
import matplotlib.pyplot as plt
import numpy as np

custom_save_dir = "enter you saved custom folder link"
json_path = os.path.join(custom_save_dir, "training_results.json")
fig_save_path = os.path.join(custom_save_dir, "training_curves.png")

with open(json_path, 'r') as f:
    results = json.load(f)

losses = results['train_losses']
epochs = len(losses)
lr_step = 15  # From your scheduler
lrs = [1e-4 * (0.5 ** (e // lr_step)) for e in range(epochs)]  # Reconstruct LR

fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(range(1, epochs+1), losses, 'b-', label='Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_title('Training Loss & LR Over Epochs')

ax2 = ax1.twinx()
ax2.plot(range(1, epochs+1), lrs, 'r--', label='LR')
ax2.set_ylabel('Learning Rate', color='r')
ax2.tick_params(axis='y', labelcolor='r')

plt.savefig(fig_save_path)
print(f"Saved to {fig_save_path}")
plt.show()